In [1]:
import math
import cmath
import random
import drawSvg as draw

C:\Users\tcich\anaconda3\lib\site-packages\drawSvg\raster.py:18: RuntimeWarning: Failed to import CairoSVG. drawSvg will be unable to output PNG or other raster image formats. See https://github.com/cduck/drawSvg#prerequisites for more details.
Original OSError: no library called "cairo" was found
cannot load library 'C:\Program Files (x86)\FAHClient\libcairo-2.dll': error 0xc1
cannot load library 'libcairo.so.2': error 0x7e
cannot load library 'libcairo.2.dylib': error 0x7e
cannot load library 'libcairo-2.dll': error 0xc1
  warnings.warn(msg, RuntimeWarning)


In [2]:
def genNeighbours(p: complex, distance: float):
    result = []
    d = complex(distance, 0)
    rot = cmath.rect(1, cmath.pi/3)
    result.append(p+d)
    for i in range(1, 3): # to generate in all directions (1,6)
        d *= rot
        result.append(p+d)
    return result

# genNeighbours(0, 1)

In [3]:
def draw_segments(d, pts, stroke='black'):
    px_size = d.width/d.renderWidth
    p = draw.Path(stroke_width=px_size, stroke=stroke, fill='none')
    start = pts[0]
    p.M(start.real, start.imag)
    for z in pts[1:]:
        p.L(z.real, z.imag)
    d.append(p)

In [4]:
def draw_points(d, pts, fill='black'):
    px_size = d.width/d.renderWidth
    for z in pts:
        c = draw.Circle(z.real, z.imag, px_size, fill=fill)
        d.append(c)

In [5]:
def draw_circles(d, pts, stroke='black'):
    px_size = d.width/d.renderWidth
    for (z,r) in pts:
        c = draw.Circle(z.real, z.imag, r, stroke_width=px_size, stroke=stroke, fill='none')
        d.append(c)

In [6]:
d = draw.Drawing(3, 3, origin='center', displayInline=False)
d.setRenderSize(w=100)

ngh = genNeighbours(0, 1)
draw_points(d, ngh+[0])

display(d)

In [7]:
class RectArea:
    def __init__(self, ld: complex, ru: complex):
        self.ld = ld
        self.ru = ru
        
    def isWithin(self, p: complex):
        return self.ld.real <= p.real <= self.ru.real and self.ld.imag <= p.imag <= self.ru.imag

# r = RectArea(0, 10+10j)
# r.isWithin(10+10j)

In [8]:
def closeEnough(p,q):
    return abs(p-q) < 0.001

def fillArea(r, gen, start):
    ps = [start]
    q = [start]
    while q:
        p = q.pop()
        pg = [p for p in gen(p, 1) if r.isWithin(p) and not any(q for q in ps if closeEnough(p,q))]
        ps += pg
        q += pg
    return ps
    
circle_mesh = fillArea(RectArea(0, 10+10j), genNeighbours, 0.5+0.5j)

In [9]:
d = draw.Drawing(10, 10, origin=(0,0), displayInline=False)
d.setRenderSize(w=600)

cs = map(lambda c: (c, 0.5), circle_mesh)

# draw_segments(d, circle_mesh, 'red')
draw_circles(d, cs)

display(d)